In [5]:
#create read_file function
def read_file(filepath):
    
    with open(filepath) as f:
        str_text = f.read()
    
    return str_text


In [6]:
# load spacy with disabling parser tager ner
import spacy
nlp = spacy.load('en',disable=['parser', 'tagger','ner'])



In [7]:
nlp.max_length=1198623

In [8]:
def seperate_punc(token_list):
    return [token.text.lower() for token in nlp(token_list) if token.text not in '\n\n \n\n\n!"-#$%&()--.*+,-/:;<=>?@[\\]^_`{|}~\t\n ']

In [9]:
# read file and seperate puntuations
d = read_file('moby_dick_four_chapters.txt')
print(d)
tokens = seperate_punc(d)

Call me Ishmael.  Some years ago--never mind how long
precisely--having little or no money in my purse, and nothing
particular to interest me on shore, I thought I would sail about a
little and see the watery part of the world.  It is a way I have of
driving off the spleen and regulating the circulation.  Whenever I
find myself growing grim about the mouth; whenever it is a damp,
drizzly November in my soul; whenever I find myself involuntarily
pausing before coffin warehouses, and bringing up the rear of every
funeral I meet; and especially whenever my hypos get such an upper
hand of me, that it requires a strong moral principle to prevent me
from deliberately stepping into the street, and methodically knocking
people's hats off--then, I account it high time to get to sea as soon
as I can.  This is my substitute for pistol and ball.  With a
philosophical flourish Cato throws himself upon his sword; I quietly
take to the ship.  There is nothing surprising in this.  If they but
knew it,

In [10]:
# organize into sequences of tokens
train_len = 25+1 # 50 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(tokens)):
    
    # Grab train_len# amount of characters
    seq = tokens[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [11]:
# import Tokenizer from keras preprocessing text
from keras.preprocessing.text import Tokenizer


Using TensorFlow backend.
OMP: Info #155: KMP_AFFINITY: Initial OS proc set respected: 0,1
OMP: Info #217: KMP_AFFINITY: decoding x2APIC ids.
OMP: Info #217: KMP_AFFINITY: cpuid leaf 11 not supported.
OMP: Info #217: KMP_AFFINITY: decoding legacy APIC ids.
OMP: Info #157: KMP_AFFINITY: 2 available OS procs
OMP: Info #158: KMP_AFFINITY: Uniform topology
OMP: Info #288: KMP_AFFINITY: topology layer "LL cache" is equivalent to "socket".
OMP: Info #192: KMP_AFFINITY: 1 socket x 1 core/socket x 2 threads/core (1 total cores)
OMP: Info #219: KMP_AFFINITY: OS proc to physical thread map:
OMP: Info #172: KMP_AFFINITY: OS proc 0 maps to socket 0 core 0 thread 0 
OMP: Info #172: KMP_AFFINITY: OS proc 1 maps to socket 0 core 0 thread 1 
OMP: Info #255: KMP_AFFINITY: pid 19783 tid 19783 thread 0 bound to OS proc set 0


In [12]:
# create Tokenizer instance
tokenizer = Tokenizer()

# fit on text for text sequences
tokenizer.fit_on_texts(text_sequences)

# convert texts to sequences
sequences = tokenizer.texts_to_sequences(text_sequences)



In [13]:
vocabulary_size = len(tokenizer.word_counts)
vocabulary_size

2709

In [14]:
# import numpy
import numpy as np

In [15]:
# convert sequences to numpy array
sequences=np.array(sequences)

In [16]:
# seperate x and y
X = sequences[:,:-1]
y = sequences[:,-1]

In [18]:
X.shape,y.shape

((11368, 25), (11368,))

In [31]:
from keras.utils import to_categorical


In [33]:
y=to_categorical(y,num_classes=vocabulary_size+1)

In [34]:
y.shape

(11368, 2710)

# Creating an LSTM based model

In [26]:
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding

In [27]:
def create_model(vocabulary_size,seq_len):
    model=Sequential()
    model.add(Embedding(vocabulary_size,seq_len,input_length=seq_len))
    model.add(LSTM(50,return_sequences=True))
    model.add(LSTM(50))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(vocabulary_size,activation='softmax'))
    model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
    model.summary()
    return model

In [28]:
X.shape[1]

25

In [29]:
from pickle import dump,load

In [30]:
model=create_model(vocabulary_size+1,X.shape[1])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 25, 25)            67750     
_________________________________________________________________
lstm_1 (LSTM)                (None, 25, 50)            15200     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_2 (Dense)              (None, 2710)              273710    
Total params: 381,960
Trainable params: 381,960
Non-trainable params: 0
_________________________________________________________________


In [40]:
model.fit(X,y,batch_size=128,epochs=250,verbose=1)

Instructions for updating:
Use tf.cast instead.


2022-05-29 20:47:41.633223: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-05-29 20:47:41.863491: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2495275000 Hz
2022-05-29 20:47:41.886743: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x5634802982f0 executing computations on platform Host. Devices:
2022-05-29 20:47:41.886869: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2022-05-29 20:47:41.888379: I tensorflow/core/common_runtime/process_util.cc:71] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Epoch 1/250


OMP: Info #255: KMP_AFFINITY: pid 56972 tid 57128 thread 2 bound to OS proc set 0
OMP: Info #255: KMP_AFFINITY: pid 56972 tid 57127 thread 3 bound to OS proc set 1
OMP: Info #255: KMP_AFFINITY: pid 56972 tid 57152 thread 4 bound to OS proc set 0


 3968/11368 [=========>....................] - ETA: 29s - loss: 7.5872 - accuracy: 0.0491

KeyboardInterrupt: 

In [27]:
# model.save("my_first_model.h5")

In [28]:
# dump(tokenizer, open('epochBIG1', 'wb'))

In [5]:
from keras.preprocessing.sequence import pad_sequences

In [6]:
def generate_word(model,tokenizer,seq_len,input_text,num_gen_words):
    output_text=[]
    for i in range(num_gen_words):
        encoded_text=tokenizer.texts_to_sequences([input_text])[0]
        padded_text=pad_sequences([encoded_text],maxlen=seq_len,truncating="pre")
        pred_word_ind=model.predict_classes(padded_text,verbose=0)[0]
        pred_word=tokenizer.index_word[pred_word_ind]
        input_text+=" "+pred_word
        output_text.append(pred_word)
    return " ".join(output_text)

In [53]:
seed_text="".join(text_sequences[0])

In [79]:
print(generate_word(model,tokenizer,25,text,55))

the night to a bit of most funeral i going unoccupied to lighted it improvements was a dark straight jet of bed it consign ourselves to time round to give me i kept a thirty years ' voyage and still how lie honest the curiosity i i 's myself a still mild face wall and


In [61]:
seed_text=" ".join(text_sequences[555])

In [62]:
seed_text

'on his legs set his feet a going and he will infallibly lead you to water if water there be in all that region should you'

In [2]:
text="what is life you ask its false hope and barrier in the"+
"middle of journey which is to reach death ,death Truth and it does not harm you"+
"it is something which give relief from suffering "

In [7]:
print(generate_word(model,tokenizer,25,text,55))

OMP: Info #255: KMP_AFFINITY: pid 5700 tid 5729 thread 2 bound to OS proc set 0
OMP: Info #255: KMP_AFFINITY: pid 5700 tid 5730 thread 3 bound to OS proc set 1
OMP: Info #255: KMP_AFFINITY: pid 5700 tid 5777 thread 4 bound to OS proc set 0


the night to a bit of most funeral i going unoccupied to lighted it improvements was a dark straight jet of bed it consign ourselves to time round to give me i kept a thirty years ' voyage and still how lie honest the curiosity i i 's myself a still mild face wall and


In [19]:
# loading
from pickle import dump,load
import pickle
with open('epochBIG1', 'rb') as handle:
    tokenizer = pickle.load(handle)
    

In [20]:
import keras
model = keras.models.load_model('my_first_model.h5')

Instructions for updating:
Colocations handled automatically by placer.


OMP: Info #255: KMP_AFFINITY: pid 19783 tid 19973 thread 1 bound to OS proc set 1
2022-06-08 13:24:39.251381: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: SSE4.1 SSE4.2 AVX AVX2 FMA
2022-06-08 13:24:39.498615: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2495285000 Hz
2022-06-08 13:24:39.524472: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x55b617640800 executing computations on platform Host. Devices:
2022-06-08 13:24:39.524561: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2022-06-08 13:24:39.526159: I tensorflow/core/common_runtime/process_util.cc:71] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


Instructions for updating:
Use tf.cast instead.
